In [2]:
# %config IPCompleter.greedy=True
import pandas as pd
import seaborn as sns
import numpy as np

import matplotlib as matplot
import matplotlib.pyplot as plt
# %matplotlib inline
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import warnings, os 
# warnings.filterwarnings("ignore")

# from keras import Sequential
# from keras.models import Model, load_model
# from keras.layers import *
# from keras.callbacks import ModelCheckpoint
# from keras import regularizers
from matplotlib import pyplot as plt

from sklearn.metrics import *
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, VotingClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from sklearn.feature_selection import RFE
from sklearn.decomposition import PCA, TruncatedSVD, PCA
from sklearn.svm import LinearSVC

import xgboost
# from mlxtend.classifier import EnsembleVoteClassifier 
import shap
import sklearn.neighbors

from deepod.models.tabular import PReNet



# Data Preparation

In [6]:
ls = [] 
directory = (r'./Dataset/CICIDS2017_cleaned_parquet/')
# for filename in os.listdir(r'./Dataset/MachineLearningCVE/'):
for filename in os.listdir(r'./Dataset/CICIDS2017_cleaned_parquet/'):
  if '.parquet' in  filename:
    print(filename)
    full_path = os.path.join(directory, filename)
    df = pd.read_parquet(full_path)
    ls.append(df)
    print(f'Shape: {df.shape}. Attack Type {df["Label"].unique()}')
    # break

DDoS-Friday-no-metadata.parquet
Shape: (221264, 78). Attack Type ['Benign', 'DDoS']
Categories (2, object): ['Benign', 'DDoS']
Bruteforce-Tuesday-no-metadata.parquet
Shape: (389714, 78). Attack Type ['Benign', 'FTP-Patator', 'SSH-Patator']
Categories (3, object): ['Benign', 'FTP-Patator', 'SSH-Patator']
Botnet-Friday-no-metadata.parquet
Shape: (176038, 78). Attack Type ['Benign', 'Bot']
Categories (2, object): ['Benign', 'Bot']
DoS-Wednesday-no-metadata.parquet
Shape: (584991, 78). Attack Type ['Benign', 'DoS slowloris', 'DoS Slowhttptest', 'DoS Hulk', 'DoS GoldenEye', 'Heartbleed']
Categories (6, object): ['Benign', 'DoS GoldenEye', 'DoS Hulk', 'DoS Slowhttptest', 'DoS slowloris', 'Heartbleed']
Benign-Monday-no-metadata.parquet
Shape: (458831, 78). Attack Type ['Benign']
Categories (1, object): ['Benign']
Infiltration-Thursday-no-metadata.parquet
Shape: (207630, 78). Attack Type ['Benign', 'Infiltration']
Categories (2, object): ['Benign', 'Infiltration']
Portscan-Friday-no-metadata.p

In [7]:
for df in ls:
  cols = df.columns.to_list()
  for i in range(len(cols)):
    cols[i] = cols[i].strip()
  df.columns = cols
df = pd.concat(ls)
pd.set_option('display.max_columns', None)
df.head(10)
# assert df.shape[1] == 79
print(df.shape)

from collections import Counter

Counter(df["Label"])

,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Fwd Packets Length Total,Bwd Packets Length Total,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Packet Length Min,Packet Length Max,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Avg Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Avg Bytes/Bulk,Fwd Avg Packets/Bulk,Fwd Avg Bulk Rate,Bwd Avg Bytes/Bulk,Bwd Avg Packets/Bulk,Bwd Avg Bulk Rate,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init Fwd Win Bytes,Init Bwd Win Bytes,Fwd Act Data Packets,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,6,3,2,0,12,0,6,6,6.0,0.00000,0,0,0.0,0.0,4.000000e+06,666666.666700,3.0,0.0,3,3,3,3.0,0.0,3,3,0,0.0,0.0,0,0,0,0,0,0,40,0,666666.687500,0.000000,6,6,6.000000,0.000000,0.000000,0,0,0,0,1,0,0,0,0,9.0,6.0,0.0,0,0,0,0,0,0,2,12,0,0,33,-1,1,20,0.0,0.0,0,0,0.0,0.0,0,0,Benign
1,6,109,1,1,6,6,6,6,6.0,0.00000,6,6,6.0,0.0,1.100917e+05,18348.623850,109.0,0.0,109,109,0,0.0,0.0,0,0,0,0.0,0.0,0,0,0,0,0,0,20,20,9174.311523,9174.311523,6,6,6.000000,0.000000,0.000000,0,0,0,0,1,1,0,0,1,9.0,6.0,6.0,0,0,0,0,0,0,1,6,1,6,29,256,0,20,0.0,0.0,0,0,0.0,0.0,0,0,Benign
2,6,52,1,1,6,6,6,6,6.0,0.00000,6,6,6.0,0.0,2.307692e+05,38461.538460,52.0,0.0,52,52,0,0.0,0.0,0,0,0,0.0,0.0,0,0,0,0,0,0,20,20,19230.769531,19230.769531,6,6,6.000000,0.000000,0.000000,0,0,0,0,1,1,0,0,1,9.0,6.0,6.0,0,0,0,0,0,0,1,6,1,6,29,256,0,20,0.0,0.0,0,0,0.0,0.0,0,0,Benign
3,6,34,1,1,6,6,6,6,6.0,0.00000,6,6,6.0,0.0,3.529412e+05,58823.529410,34.0,0.0,34,34,0,0.0,0.0,0,0,0,0.0,0.0,0,0,0,0,0,0,20,20,29411.765625,29411.765625,6,6,6.000000,0.000000,0.000000,0,0,0,0,1,1,0,0,1,9.0,6.0,6.0,0,0,0,0,0,0,1,6,1,6,31,329,0,20,0.0,0.0,0,0,0.0,0.0,0,0,Benign
4,6,3,2,0,12,0,6,6,6.0,0.00000,0,0,0.0,0.0,4.000000e+06,666666.666700,3.0,0.0,3,3,3,3.0,0.0,3,3,0,0.0,0.0,0,0,0,0,0,0,40,0,666666.687500,0.000000,6,6,6.000000,0.000000,0.000000,0,0,0,0,1,0,0,0,0,9.0,6.0,0.0,0,0,0,0,0,0,2,12,0,0,32,-1,1,20,0.0,0.0,0,0,0.0,0.0,0,0,Benign
5,6,1022,2,0,12,0,6,6,6.0,0.00000,0,0,0.0,0.0,1.174168e+04,1956.947162,1022.0,0.0,1022,1022,1022,1022.0,0.0,1022,1022,0,0.0,0.0,0,0,0,0,0,0,40,0,1956.947144,0.000000,6,6,6.000000,0.000000,0.000000,0,0,0,0,1,0,0,0,0,9.0,6.0,0.0,0,0,0,0,0,0,2,12,0,0,32,-1,1,20,0.0,0.0,0,0,0.0,0.0,0,0,Benign
6,6,4,2,0,12,0,6,6,6.0,0.00000,0,0,0.0,0.0,3.000000e+06,500000.000000,4.0,0.0,4,4,4,4.0,0.0,4,4,0,0.0,0.0,0,0,0,0,0,0,40,0,500000.000000,0.000000,6,6,6.000000,0.000000,0.000000,0,0,0,0,1,0,0,0,0,9.0,6.0,0.0,0,0,0,0,0,0,2,12,0,0,32,-1,1,20,0.0,0.0,0,0,0.0,0.0,0,0,Benign
7,6,42,1,1,6,6,6,6,6.0,0.00000,6,6,6.0,0.0,2.857143e+05,47619.047620,42.0,0.0,42,42,0,0.0,0.0,0,0,0,0.0,0.0,0,0,0,0,0,0,20,20,23809.523438,23809.523438,6,6,6.000000,0.000000,0.000000,0,0,0,0,1,0,0,0,1,9.0,6.0,6.0,0,0,0,0,0,0,1,6,1,6,32,256,0,20,0.0,0.0,0,0,0.0,0.0,0,0,Benign
8,6,4,2,0,37,0,31,6,18.5,17.67767,0,0,0.0,0.0,9.250000e+06,500000.000000,4.0,0.0,4,4,4,4.0,0.0,4,4,0,0.0,0.0,0,0,1,0,0,0,40,0,500000.000000,0.000000,6,31,22.666666,14.433757,208.333328,0,1,0,0,1,0,0,0,0,34.0,18.5,0.0,0,0,0,0,0,0,2,37,0,0,946,-1,1,20,0.0,0.0,0,0,0.0,0.0,0,0,Benign
9,6,3,2,0,37,0,31,6,18.5,17.67767,0,0,0.0,0.0,1.230000e+07,666666.666700,3.0,0.0,3,3,3,3.0,0.0,3,3,0,0.0,0.0,0,0,1,0,0,0,40,0,666666.687500,0.000000,6,31,22.666666,14.433757,208.333328,0,1,0,0,1,0,0,0,0,34.0,18.5,0.0,0,0,0,0,0,0,2,37,0,0,980,-1,1,20

(2313810, 78)


Counter({'Benign': 1977318,
         'DoS Hulk': 172846,
         'DDoS': 128014,
         'DoS GoldenEye': 10286,
         'FTP-Patator': 5931,
         'DoS slowloris': 5385,
         'DoS Slowhttptest': 5228,
         'SSH-Patator': 3219,
         'PortScan': 1956,
         'Web Attack � Brute Force': 1470,
         'Bot': 1437,
         'Web Attack � XSS': 652,
         'Infiltration': 36,
         'Web Attack � Sql Injection': 21,
         'Heartbleed': 11})

### Sampling and Label Encodings

In [8]:
#### Sample out benign data to a ratio of Benign : Malicious = 4:10
# print(f'df.shape {df.shape} before sampling out most of benign data')
# df = pd.concat([df[df['Label'] != 'BENIGN'], df[df['Label'] == 'BENIGN'].sample(frac=.1, random_state=42)]) # 20% of total is Malicious; Use only a sample of the BENIGN data
# print(f'df.shape {df.shape} after sampling out most of benign data')

#### Sample out Malicious data such that it would become an outlier
print(f'df.shape {df.shape} Before Sampling Out')
# df = pd.concat([df[df['Label'] == 'Benign'], df[df['Label'] != 'Benign'].sample(frac=0.1, random_state=42)]) 
print(f'df.shape {df.shape} after Sampling Out')

# #### filter out malicious data
# print(f'df.shape {df.shape} Before Sampling Out')
# df = df[df['Label'] == 'BENIGN']
# print(f'df.shape {df.shape} after Sampling Out')


df.shape (2313810, 78) Before Sampling Out
df.shape (2313810, 78) after Sampling Out


### Data Cleaning and Normalization

In [9]:
import gc 
gc.collect()
precleaned_df = df.copy()

483

In [10]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
df = precleaned_df.copy()
# labels = df['Label'].copy()
# strLabels = df['strLabel'].copy()
# df.drop(columns='Label', axis=1, inplace=True)
# df.drop(columns='strLabel', axis=1, inplace=True)


# Find and handle null values
null_counts = df.isnull().sum()
# Print the number of null values
print(f"{null_counts.sum()} null entries have been found in the dataset\n")
# Drop null values
df.dropna(inplace=True)          # or df_data = df_data.dropna()

# Find and handle duplicates
duplicate_count = df.duplicated().sum()
# Print the numer of duplicate entries
print(f"{duplicate_count} duplicate entries have been found in the dataset\n")
# Remove duplicates
df.drop_duplicates(inplace=True)  # or df_data = df_data.drop_duplicates()
# Display relative message
print(f"All duplicates have been removed\n")

# Reset the indexes
df.reset_index(drop=True, inplace=True)

# Inspect the dataset for categorical columns
print("Categorical columns:",df.select_dtypes(include=['object']).columns.tolist(),'\n')

# print(np.any(np.isnan(df)))
# print(np.any(np.isfinite(df)))

# scaler = StandardScaler()
# scaler = MinMaxScaler()
# scaler = RobustScaler()

# normalized_data = scaler.fit_transform(df[:])
normalized_df = pd.DataFrame(df, columns=df.columns)

# Last column turn to binary
# df.iloc[:, -1] = df.iloc[:, -1].apply(lambda x: 1 if x != 0 else 0).astype(int)
# normalized_df['Label'] = labels
# df['strLabel'] = strLabels
print(normalized_df.shape)



0 null entries have been found in the dataset

82004 duplicate entries have been found in the dataset

All duplicates have been removed

Categorical columns: ['Label'] 

(2231806, 78)


In [11]:
# 1.2.3 - Set up binary configuration

# Extract features as X
X = df.copy()
X = X.drop('Label', axis=1)

# Extract target as y
y = df['Label'].copy()

# Binarize labels
y = y.map({'Benign': 0}).fillna(1).astype(int)
y

0          0
1          0
2          0
3          0
4          0
          ..
2231801    0
2231802    0
2231803    0
2231804    0
2231805    0
Name: Label, Length: 2231806, dtype: int64

In [12]:

# X_train, X_temp, y_train, y_temp = train_test_split(normalized_df.drop(['Label'], axis=1), normalized_df['Label'], test_size=0.20, random_state=42)
# X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.50, random_state=42)

In [13]:
def extractAllSets(X,y,p_train,p_val,p_test,random_state, shuffle=True):

    X_train, X_temp, y_train, y_temp = train_test_split(X,y,                
                                                        stratify=y,
                                                        test_size=(1.0 - p_train),
                                                        random_state=random_state,
                                                        shuffle=shuffle)
                                                        # Enable/disable shuffling
    fraction = p_test / (p_val + p_test)
    X_val, X_test, y_val, y_test = train_test_split(X_temp,y_temp,stratify=y_temp,
                                                      test_size=fraction,
                                                      random_state=random_state,
                                                      shuffle=shuffle)
                                                      # Enable/disable shuffling
    
    return X_train, X_val, X_test, y_train, y_val, y_test   
  
X_train, X_val, X_test, y_train, y_val, y_test = extractAllSets(X,y,0.5,0.1,0.4,random_state = 4022)
original_data = [X_train, X_val, X_test, y_train, y_val, y_test]

robust_scaler = RobustScaler()
#Validation for 
X_train_scaled = pd.DataFrame(robust_scaler.fit_transform(X_train), columns=X_train.columns)
X_val_scaled   = pd.DataFrame(robust_scaler.transform(X_val),       columns=X_val.columns)
X_test_scaled  = pd.DataFrame(robust_scaler.transform(X_test),      columns=X_test.columns)
y_train = y_train.reset_index(drop = True)
y_val = y_val.reset_index(drop = True)
y_test = y_test.reset_index(drop = True)


In [14]:
# Assuming df is your DataFrame and you've already executed the train_test_split as shown earlier
X_train_tensor = torch.tensor(X_train_scaled.values, dtype=torch.float)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float)

X_val_tensor = torch.tensor(X_val_scaled.values, dtype=torch.float32)  # Using X_test as validation data
y_val_tensor = torch.tensor(y_val.values, dtype=torch.float32)  

X_test_tensor = torch.tensor(X_test_scaled.values, dtype=torch.float32)  # Using X_test as validation data
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32)  

# X_full_tensor = torch.tensor(X.values, dtype=torch.float32)  # Using X_test as validation data
# y_full_tensor = torch.tensor(y.values, dtype=torch.float32)  

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=1024, shuffle=True)

val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
val_loader = DataLoader(val_dataset, batch_size=1024, shuffle=False)

test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=1024, shuffle=False)

# full_dataset = TensorDataset(X_full_tensor, y_full_tensor)
# full_loader = DataLoader(full_dataset, batch_size=128, shuffle=False)




In [15]:
def get_score(y_true, y_pred):
  accuracy = accuracy_score(y_true, y_pred)
  precision = precision_score(y_true, y_pred)
  recall = recall_score(y_true, y_pred)
  f1 = f1_score(y_true, y_pred)

  print(f"Accuracy: {accuracy:.4f}")
  print(f"Precision: {precision:.4f}")
  print(f"Recall: {recall:.4f}")
  print(f"F1 Score: {f1:.4f}")

# PReNet

In [16]:
clf = PReNet()
clf.fit(X_train, y=y_train)

Start Training...
ensemble size: 1


AssertionError: Torch not compiled with CUDA enabled

In [5]:
X_train

NameError: name 'X_train' is not defined